In [21]:
import torch
import os
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from timeit import default_timer as timer 
import pickle
import pandas as pd

pd.set_option('display.max_columns', 500)

In [22]:
with open('churn_model.pcl', 'rb') as file:
    deserialized_object = pickle.load(file)

In [23]:
train_df = deserialized_object[1][1]
train_num = deserialized_object[1][2]
train_dpi = deserialized_object[1][3]

test_df = deserialized_object[2][1]
test_num = deserialized_object[2][2]
test_dpi = deserialized_object[2][3]

In [26]:
train_num.dtypes

bnum             object
call_cnt_out    float64
call_cnt_in     float64
call_dur_out    float64
call_dur_in     float64
cnt_sms_out     float64
cnt_sms_in      float64
abon_id         float64
dtype: object

In [20]:
deserialized_object

({'1': 'TRAIN Datasets', '2': 'TEST Datasets'},
 ({'1': 'TRAIN_MODEL_FE', '2': 'TRAIN_MODEL_B_NUM', '3': 'TRAIN_MODEL_DPI'},
          Ama_rchrgmnt_sum_max_mnt1  content_clc_mea_mnt1  content_cnt_max_mnt1  \
  0                               0                   0.0             13.841608   
  1                               0                   0.0             11.361162   
  2                               0                   0.0             10.269117   
  3                               0                   0.0              9.974412   
  4                               0                   0.0              6.749902   
  ...                           ...                   ...                   ...   
  149995                          0                   0.0              8.333536   
  149996                          0                   0.0             14.038728   
  149997                          0                   0.0             13.225565   
  149998                          0          